In [ ]:
import uproot
import numpy as np
import pandas as pd
import awkward as ak
import matplotlib.pyplot as plt

In [ ]:
particles = uproot.open("/Users/andreas/cern/source/acts/acts/dev3/colliderml_output/particles_simulated.root")
particles = particles["particles"].arrays(library="ak")
particles = ak.to_dataframe(particles, how="inner")

particles_selected = uproot.open("/Users/andreas/cern/source/acts/acts/dev3/colliderml_output/particles_selected.root")
particles_selected = particles_selected["particles"].arrays(library="ak")
particles_selected = ak.to_dataframe(particles_selected, how="inner")

hits = uproot.open("/Users/andreas/cern/source/acts/acts/dev3/colliderml_output/hits.root")
hits = hits["hits"].arrays(library="ak")
hits = ak.to_dataframe(hits, how="inner")

In [ ]:
particles

In [ ]:
particles_selected

In [ ]:
hits

In [ ]:
ref_particles_selected = particles[
    (particles["vz"].abs() < 1000) &
    (np.linalg.norm(particles[["vx","vy"]].values,axis=1) < 24) &
    (particles["pt"] >= 1) &
    (particles["q"] != 0) &
    (particles["eta"].abs() < 3) &
    (particles["number_of_hits"] >= 6)
]

types, counts = np.unique(ref_particles_selected["particle_type"], return_counts=True)

plt.bar([str(t) for t in types], counts);
plt.gca().tick_params(axis='x', labelrotation=90)

ref_particles_selected

In [ ]:
plt.hist2d(ref_particles_selected["eta"], ref_particles_selected["number_of_hits"], bins=(30, 20))
plt.xlabel(r"$\eta$")
plt.ylabel("number of hits")

pass

In [ ]:
types, counts = np.unique(particles_selected["particle_type"], return_counts=True)

plt.bar([str(t) for t in types], counts);
plt.gca().tick_params(axis='x', labelrotation=90)

particles_selected

In [ ]:
plt.hist2d(particles_selected["eta"], particles_selected["number_of_hits"], bins=(30, 20))
plt.xlabel(r"$\eta$")
plt.ylabel("number of hits")

pass

In [ ]:
plt.scatter(hits["tz"], np.linalg.norm(hits[["tx","ty"]].values,axis=1), s=0.5, color="C0", zorder=0)

hits_selected = particles_selected.set_index(["event_id", "particle_id"]).join(hits.set_index(["event_id", "particle_id"]), how="inner")

plt.scatter(hits_selected["tz"], np.linalg.norm(hits_selected[["tx","ty"]].values,axis=1), s=0.5, color="C1", zorder=1)

n = 0
for (event_id, particle_id), particle_hits in hits_selected.groupby(["event_id", "particle_id"]):
    if n > 100:
        break
    n += 1

    particle_hits["distance_origin"] = np.linalg.norm(particle_hits[["tx","ty","tz"]].values,axis=1)
    particle_hits.sort_values(by="distance_origin")

    plt.plot(particle_hits["tz"], np.linalg.norm(particle_hits[["tx","ty"]].values,axis=1), "--", color="C2", zorder=2)